# Academic Achievement Gap Analysis: 
## Data Cleaning, Wrangling, and Transformation

## Introduction 

Upon graduating from university with a degree in International Relations, I joined an alternative teacher certification program called Teach for America (TFA). Why become an educator after writing a senior thesis on U.S. anti-narcotics policies in the 1980s? Because of TFA's focus on social justice. 

TFA's primary purpose is to close the "achievement gap" and end educational inequity - as it makes very clear in the "About Us" section of the TFA website: https://www.teachforamerica.org/about-us. What this means in practice is that TFA sends its teachers ("corps members") to teach in low-income schools across the country - generally where at least 80% of students at a school receive Title I funding. Such funds are provided to school districts to help support students living below the poverty line (as regulated by the "Every Student Succeeds Act").

Like most corps members, I unquestioningly accepted TFA's premise regarding the relationship between poverty and school performance. But now I would like to explore the idea in greater depth, using data from the U.S. Department of Education's EDFacts Initiative, U.S. Census Bureau's Small Area Income and Poverty Estimates Program, and the U.S. Department of Education's Local Education Agency (School District) Universe Survey Data to answer the following question: Do students living in poorer communities perform worse on end-of-year reading and math tests? 

This notebook contains the data cleaning, wrangling and transformation that was required before this analysis could be completed. To see my final storytelling project, click <a href="https://nbviewer.jupyter.org/gist/TGasinski/2b2be58c17860f5868fbb084142a550c">here</a>.   

## Import Packages and Data

I will begin by importing all relevant packages and my first dataset - the district-level data on state math assessment results for the 2011-2012 school year. I can use the pd.set_option() and .head() functions to view a section of the data.

__Data Source:__ https://catalog.data.gov/dataset/consolidated-state-performance-report-201112/resource/4da8f9de-d265-4e2f-b321-f180052df3f7

In [1]:
#import all needed packages
import pandas as pd
import csv as csv
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import scipy


#read in first set of data
df_math_org= pd.read_csv(r'C:\Users\t_gas\Desktop\Data Portfolio\Data Storytelling_Exploring the Academic Achievement Gap\2011_2012_School_Data\Test_Scores\Math_District.csv') 

By reading the EdFacts data documentation, I can learn more about what each column header signifies. For example, the "01", "02", "03", etc. represent each grade level within that county.

I know that I will want to compare data across counties but also across states. So before continuing, I will create a list of unique state names that I can use later in my analysis

In [2]:
#create list of state names to use later in analysis
state_names = []
 
for x in df_math_org.iloc[:,0]:
    if x in state_names:
        continue
    else:
        state_names.append(x)  

I know that I will only explore the proficiency levels of an entire county, rather than a specific grade within that county. From the data documentation, I know that "00" represents all grades, 1-8, within the county. So for the sake of my computer's RAM, I will create a smaller dataframe with only the relevant columns. 

In [3]:
#create smaller dataframe with only necessary columns
df_math_all_grades=df_math_org[['STNAM', 'FIPST', 'LEAID', 'leanm11', 'ALL_MTH00pctprof_1112', 'MAM_MTH00pctprof_1112', 'MAS_MTH00pctprof_1112', 'MBL_MTH00pctprof_1112', 'MHI_MTH00pctprof_1112', 'MTR_MTH00pctprof_1112', 'MWH_MTH00pctprof_1112', 'F_MTH00pctprof_1112', 'M_MTH00pctprof_1112', 'CWD_MTH00pctprof_1112', 'ECD_MTH00pctprof_1112', 'LEP_MTH00pctprof_1112', 'HOM_MTH00pctprof_1112', 'MIG_MTH00pctprof_1112']].copy()  

Because all of the values in the column "ALL_MTH00pctprof_1112" represent percentages of students, I should be able to convert them into numeric values. However, after examining the dataframe using .info(), it appears that all the values are saved as objects, not numbers (int64). This could mean that not all the saved values are numbers. In fact, as I examined the original dataframe, I already spotted data that will not be usable in its current state (e.g. the Math percent proficiency for Alabama Youth Services school was "30-39"). 

## Cleaning Data

By creating a set of values that cannot be converted, I can identify all the unique errors within the column. I will then create a for loop to fix the types of errors presented in the data. 

In [4]:
#create list of unique errors within dataframe
errors = set()

for i in df_math_all_grades['MBL_MTH00pctprof_1112']:
    try: 
        int(i)
    except: 
        errors.add(i)

After executing print(errors), I have a clearer picture of what types of erroneous data exist in the dataframe and can decide how best to clean the data. 

When given two values (e.g. "40-44"), I will replace the two values with the values' mean. For all data including letters before a value, I will simply drop the letters. According to the documentation, "PS" means that the data has been removed to protect student privacy. In these instances, I will substitute "PS" with "NaN." 

In [5]:
#create for loop to clean up each type of error found earlier in the dataframe
for x,i in df_math_all_grades['MBL_MTH00pctprof_1112'].items():
    i = (str(i))
    if i in errors:
        if i == "nan":
            continue
        elif "-" in i:
            numbers = list(map(int, i.split("-")))
            replacement = np.mean(numbers)
        elif "PS" in i: 
            replacement = np.nan
        elif "n/a" in i: 
            replacement = np.nan
        elif re.search(r"[A-Za-z]", i): 
            working_list = list(i)
            numbers = []
            for j in working_list:
                if j.isnumeric():
                    numbers.append(j)
            replacement = int("".join(numbers))
        df_math_all_grades.iloc[x, 7] = replacement

I can perform a quick check to ensure that no errors were overlooked, by printing the output of my simple loop using print(check).

In [6]:
#test to make sure no more errors remain 
check = set()

for i in df_math_all_grades['MBL_MTH00pctprof_1112']:
    try: 
        int(i)
    except: 
        check.add(i)

Next, I will create a function in order to clean all the columns within my dataset. But before designing the function, I create two lists: a list of all column names within my dataset, and a list of column names that I do not want my function to clean (as the values represented are state and county names, along with county IDs, rather than test scores).

In [7]:
#create lists of values to ignore in function
column_names_math = list(df_math_all_grades)
ignore = ["STNAM", "FIPST", "LEAID", "leanm11"]

I will use these lists in a function, which will clean up all the errors I identified earlier. 

In [8]:
#write function to clean columns within dataframe
def cleaning_columns(df,column,order_number): 
    errors = set()
    for i in column:
        try:
            int(i)
        except:
            errors.add(i)

    for x,i in column.items():
        i = (str(i))
        if i in errors:
            if i == "nan":
                continue
            elif "-" in i:
                numbers = list(map(int, i.split("-")))
                replacement = np.mean(numbers)
            elif "PS" in i:
                replacement = np.nan
            elif "n/a" in i:
                replacement = np.nan
            elif re.search(r"[A-Za-z]", i):
                working_list = list(i)
                numbers = []
                for j in working_list:
                    if j.isnumeric():
                        numbers.append(j)
                replacement = int("".join(numbers))
            
            df.iloc[x, order_number] = replacement
            
    return(df)

#run function
for i in range(0,len(column_names_math)):
    if column_names_math[i] in ignore:
        continue
    else:
        name = column_names_math[i]
        cleaning_columns(df_math_all_grades, df_math_all_grades[name], i)

I create a second function that will allow me to quickly check that all of my data was cleaned correctly, by creating a function and then viewing the results using: print(check), as I did manually after the first column had been cleaned.

In [9]:
#create function to ensure data was cleaned correctly
def check_data(column):
    check = set()
    for i in column: 
        try: 
            int(i)
        except: 
            check.add(i)

#run function
for i in range(0, len(column_names_math)): 
    if column_names_math[i] in ignore: 
        continue
    else: 
        name = column_names_math[i]
        check_data(df_math_all_grades[name])

Now, I can use the same functions to clean my second data set, which contains the district-level data on state reading assessment results for the 2011-2012 school year. 

__Data Source:__ https://catalog.data.gov/dataset/consolidated-state-performance-report-201112/resource/fd3382ec-8a1d-4372-930c-0e4f72db5037

In [10]:
#read in second set of data and create unique list of column names 
df_reading_org= pd.read_csv(r'C:\Users\t_gas\Desktop\Data Portfolio\Data Storytelling_Exploring the Academic Achievement Gap\2011_2012_School_Data\Test_Scores\Reading_District.csv')
df_reading_all_grades=df_reading_org[['STNAM', 'FIPST', 'LEAID', 'leanm11', 'ALL_RLA00pctprof_1112', 'MAM_RLA00pctprof_1112', 'MAS_RLA00pctprof_1112', 'MBL_RLA00pctprof_1112', 'MHI_RLA00pctprof_1112', 'MTR_RLA00pctprof_1112', 'MWH_RLA00pctprof_1112', 'F_RLA00pctprof_1112', 'M_RLA00pctprof_1112', 'CWD_RLA00pctprof_1112', 'ECD_RLA00pctprof_1112', 'LEP_RLA00pctprof_1112', 'HOM_RLA00pctprof_1112', 'MIG_RLA00pctprof_1112']].copy()  
column_names_reading = list(df_reading_all_grades) 
ignore_reading = ["STNAM", "FIPST", "LEAID", "leanm11"]

In [11]:
#run function to clean dataset
for i in range(0,len(column_names_reading)):
    if column_names_reading[i] in ignore_reading:
        continue
    else: 
        name = column_names_reading[i]
        cleaning_columns(df_reading_all_grades, df_reading_all_grades[name], i)

In [12]:
#run function to ensure data was cleaned correctly
for i in range(0, len(column_names_reading)): 
    if column_names_reading[i] in ignore: 
        continue
    else: 
        name = column_names_reading[i]
        check_data(df_reading_all_grades[name])

Using .dtypes, I can see that almost all the rows in both dataframes have been stored as objects. This means that while the dataframes are clean, they are still not yet ready to be analyzed. In order to further explore the data, I need to convert the information into numeric values. 

To do so, I will build a function that ignores the first four rows (which contain text information, such as county name) and converts the rest of the column information (which contain percentages) into numeric values. 

In [13]:
#create function to convert data into numeric values, then use .dtypes 
#to check that all values were converted correctly

def numeric (df, column):
    for column in df: 
        df[column] = df[column].apply(pd.to_numeric, errors = 'ignore')
    return(df)

for i in range(0, len(column_names_math)):
    if column_names_math[i] in ignore: 
        continue
    else: 
        name = column_names_math[i]
        numeric(df_math_all_grades, df_math_all_grades[name])

In [14]:
#run function and check value types using df.dtypes
for i in range(0, len(column_names_reading)):
    if column_names_reading[i] in ignore: 
        continue
    else: 
        name = column_names_reading[i]
        numeric(df_reading_all_grades, df_reading_all_grades[name])

Now that all the data has been converted into a numeric value, I can continue to explore the dataframes. I will begin by examining what percentage of my data contains NaN values, by creating a function.

In [15]:
#create function to identify percent of data that contains nan values
def percent_valid(column): 
    length = len(column)
    nan = column.isnull().sum()
    return ((length-nan)/length)*100

If I were to run this function on both of my dataframes (e.g.: "df_math_all_grades.apply(percent_valid, axis=0)") I could see exactly what percentage NaN values existed in each column

## Manipulate Dataframes

I will create two smaller dataframes including only the columns with meaningful data (this will include the state name, county name, and total percent proficiency for both the Math and Reading state assessment results). Because these simplified dataframes contain a lot of the same information, I can then combine them into a single dataframe. 

In [16]:
#merge dataframes
df_math_only_totals = df_math_all_grades[['STNAM', 'LEAID', 'leanm11', 'ALL_MTH00pctprof_1112']].copy()
df_reading_only_totals = df_reading_all_grades[['STNAM', 'LEAID', 'leanm11', 'ALL_RLA00pctprof_1112']].copy()
df_all_states = pd.merge(df_math_only_totals, df_reading_only_totals, how = 'left')

Next, I will import a dataset which shows the poverty rates per country in 2011 and 2012. 

To measure poverty, the Census Bureau assigns each family member one out of 48 possible "poverty thresholds." These thresholds vary by the size of the family and the age of the family members, but the thresholds themselves do not vary geographically. The thresholds are also updated annually for inflation. The Census Bureau then calculates the total family income, by adding up the income of all family members that live together. 

If the total family income is less than the poverty threshold for that family, that family and every individual in it is considered to be inpoverty. If the income is equal to or greater than the poverty threshold, that family is not considered to be in poverty. For a complete explanation of how the Census Bureau measures poverty, see: https://www.census.gov/topics/income-poverty/poverty/guidance/poverty-measures.html

__Data Sources:__ https://www.census.gov/data/datasets/2011/demo/saipe/2011-state-and-county.html and https://www.census.gov/data/datasets/2012/demo/saipe/2012-state-and-county.html 

(Note: It appears that the U.S. Census Department restructured its datafiles. When I originally downloaded the dataset, the 2011 and 2012 poverty estimates were combined into one file. Now, they exist in two separate files.) 

In [17]:
#read in third set of data
df_poverty_org= pd.read_csv(r'C:\Users\t_gas\Desktop\Data Portfolio\Data Storytelling_Exploring the Academic Achievement Gap\2011_2012_School_Data\Poverty_County\Poverty_Estimates_County.csv')
pd.set_option("display.max_columns", 500)

After examining the first few rows of the dataset, I immediately notice that the county name is followed by the state abbreviation in parantheses. This abbreviation will have to be migrated to a new column, so that I can merge the poverty data with my other dataframes.

In [18]:
#split column
df_poverty_org['County'], df_poverty_org['State_2'] = df_poverty_org['State / County Name'].str.split('(', 1).str
df_poverty_org['State_2'] = df_poverty_org['State_2'].str.replace(r"\.*\)","")

Now I will create a smaller database that contains only the information about children ages 5-17, whose families are in poverty. This is the information provided by the Census Bureau to the U.S. Department of Education, who uses such information to determine the correct distribution of Title 1 funds. 

In [19]:
#create smaller dataframe with only necessary columns
df_poverty = df_poverty_org[['Year', 'State / County Name', 'County', 'State_2', 'Ages 5 to 17 in Families in Poverty Percent']].copy()

.info() provides more information about how the poverty information is stored. Thankfully, the percentages of children living in families below the poverty line have already been saved as float64 objects.  

By running "df_poverty.isnull().sum(), I can see how many NaN values are within each row of the dataframe. The NaN values correctly saved and the total (104 for the column "State_2") is fairly low and will not invalidate the data. 

The American Community Survey (where this poverty data is derived from) occurs once every calendar year. However, test scores are measured at the end of a school year (which lasts between two calendar years). Therefore, I will take the mean of the 2011 and 2012 poverty percentages for each county. To do this, I need to reshape the dataframe.

Originally, I grouped the data based on the column name "County." However, this resulted in a significant amount of lost data (because county names repeated within different states). To solve this problem, I grouped by the column "State / County Name," which included both the state and county name in one column. 

In [20]:
#create new dataframe grouped by "State/County Name"
df_poverty_county_means_org = df_poverty.groupby(['State / County Name'])['Ages 5 to 17 in Families in Poverty Percent'].mean()
df_poverty_county_means = df_poverty_county_means_org.to_frame()
df_poverty_county_means_indexed = df_poverty_county_means.reset_index()

I will now have to separate the county and state into two separate columns like I did before, creating a new dataframe using .groupby() and .mean()

In [21]:
#split columns
df_poverty_county_means_indexed['County'], df_poverty_county_means_indexed['State'] = df_poverty_county_means_indexed['State / County Name'].str.split('(', 1).str
df_poverty_county_means_indexed['State'] = df_poverty_county_means_indexed['State'].str.replace(r"\.*\)","")

The dataframe above ("df_poverty_county_means_indexed") is now organized by county, while my state assessment dataframe ("df_all_states") is organized by school district. Unfortunately I cannot directly merge the two dataframes, because not every school district directly corresponds to a county. 

To correctly filter and subsequently merge the dataframes, I will use the same logic as the Census Bureau uses when determining Title 1 funding for school districts. The Bureau creates a subset of regular school districts from the larger NCES CCD Local Education Agency universe - "regular" being defined as school districts which are geographically defined. (See https://nces.ed.gov/programs/edge/geographicDistrictBoundary.aspx for more information.) 

__Data Source:__ https://nces.ed.gov/ccd/pubagency.asp (2011-2012)

In [22]:
#read in fourth set of data
df_district_county_org = pd.read_excel(r'C:\Users\t_gas\Desktop\Data Portfolio\Data Storytelling_Exploring the Academic Achievement Gap\2011_2012_School_Data\School District Info\District_Data.xlsx')

After importing the original dataframe, I create a copy with just the relevant columns. I then rename certain columns within the smaller dataframe, to better correspond with the information in my state assessment scores dataframes. 

In [23]:
#create a smaller dataframe with only necessary columns
df_district_county = df_district_county_org[['LEAID', 'STID', 'NAME', 'LSTATE', 'TYPE','CONAME', 'BIEA']].copy()
#rename columns
df_district_county.rename(columns = {'NAME': 'leanm11', 'CONAME': 'County'}, inplace=True)

I will now create a smaller database, where only rows with the column "Type" = 1 or the column "BIEA" = 2 are preserved. This is based on the Census Bureau's calculations, as mentioned above.

In [24]:
#filter dataframe 
df_regular_district_county = df_district_county[df_district_county['TYPE'] == 1]
df_regular_district_county_final = df_regular_district_county[df_regular_district_county['BIEA'] == 2]

Before merging the state assessment dataframe with the regular districts dataframe, I will compare how many rows exist in each dataframe using .info(). In doing so, I see that there are approximately 3,000 more entries in the state assessment dataframe. This means that when I merge the two dataframes, I should use an outer join to ensure that these 3,000 rows are not dropped.

In [25]:
#create a dictionary of state abbreviations
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Bureau Of Indian Affairs' : 'BOIA'
}

In [26]:
#merge dataframes
df_scores_districts_counties_org = pd.merge(df_all_states, df_regular_district_county_final, on = 'LEAID', how = 'outer')

.info() allows me to examine the new merged dataframe and check that no entries have been dropped.

I am now almost ready to merge the two datasets. I will have to add one final column to "df_scores_districts_counties," 
#which contains state abbreviations.

In [27]:
#add column to dataframe
df_scores_districts_counties_org['STNAM'] = df_scores_districts_counties_org['STNAM'].str.title() 
df_scores_districts_counties_org['State'] = df_scores_districts_counties_org['STNAM'].map(state_abbreviations)

#Now that the state abbreviations have been added, I will remove unnecessary columns and reorder the remaining columns.


In [28]:
#remove unnecessary columns and reorder remaining columns
df_scores_districts_counties = df_scores_districts_counties_org[['County', 'State', 'LEAID', 'leanm11_x', 'leanm11_y', 'ALL_MTH00pctprof_1112', 'ALL_RLA00pctprof_1112']].copy()
df_scores_districts_counties['County'] = df_scores_districts_counties['County'].str.title() 
df_scores_districts_counties['State/County'] = df_scores_districts_counties['County'] + ' ' + df_scores_districts_counties['State'].map(str)

I will use the same process as above with the poverty database.

In [29]:
#remove unnecessary columns and reorder remaining columns
df_poverty_county_means_indexed_small = df_poverty_county_means_indexed[['County', 'State', 'Ages 5 to 17 in Families in Poverty Percent']].copy()
df_poverty_county_means_indexed_small['State/County'] = df_poverty_county_means_indexed_small['County'] + '' + df_poverty_county_means_indexed_small['State'].map(str)
df_poverty_county_means_indexed_small = df_poverty_county_means_indexed_small.sort_values(by=['State'], ascending = False)

In [30]:
#create dictionary of poverty levels per county
df_poverty_dictionary = df_poverty_county_means_indexed_small[['Ages 5 to 17 in Families in Poverty Percent', 'State/County']].copy()
poverty_dictionary = df_poverty_dictionary.set_index('State/County').T.to_dict('list')

In [31]:
#create new column in dataframe
df_scores_districts_counties['Ages 5 to 17 in Families in Poverty Percent'] = np.nan

#add in values from dictionary
for j in poverty_dictionary:
    df_scores_districts_counties.loc[df_scores_districts_counties['State/County']==j,['Ages 5 to 17 in Families in Poverty Percent']] = poverty_dictionary[j] 

In [32]:
#after checking the amount of nan data using .info(), I create final dataframe 
df_final = df_scores_districts_counties[['LEAID', 'County', 'State', 'State/County', 'leanm11_x', 'ALL_MTH00pctprof_1112', 'ALL_RLA00pctprof_1112', 'Ages 5 to 17 in Families in Poverty Percent']].copy()
df_final.rename(columns = {'leanm11_x':'School District' }, inplace=True) 
df_final = df_final.dropna(axis=0, how='any')

In [33]:
#export final dataset as .csv file

df_final.to_csv("df_final.csv")